In [165]:
import pandas as pd
import numpy as np
import json
import os
import urllib3
import csv
import logging
import re
import uuid
import shortuuid
from pathlib import Path
from idmc_utils import generate_taskflow, package_import
from urllib.parse import quote

# Set the pandas options
pd.set_option('display.max_columns', None)

# Initialise the job

In [166]:
# Initialise the log file
logging.basicConfig(
    filename='logs/console.log',
    level=logging.DEBUG,
    format='%(asctime)s:%(levelname)s:%(message)s',
)

In [167]:
# Read the config file
logging.info('Reading the config file')
with open('config/config.json', 'r') as infile:
    config = json.load(infile)


# Read the Inputs

In [168]:
# Read the execution plans
logging.info('Reading the execution plans')
dfPlans = pd.read_csv('in/plans.csv', dtype='str', encoding='utf-8', na_filter=False)
dfPlans['plan_step_order'] = dfPlans['plan_step_order'].astype(int)

#TODO remove the below filter once done with testing
test_plans = [
    'Echo Employee Snapshot Oracle R12.1.3', 
    'Echo GL Balance Refresh', 
    'Echo Account Dimension', 
    'Human Resources - Oracle R1213 - Flexfield', 
    'Common-LoadDayDimension Universal',
    'Process_Quality_OracleR1213'
    ]
dfPlans = dfPlans[dfPlans['plan_name'].isin(test_plans)]

# Read the parameter files
dfParams = pd.read_csv('in/params.csv', dtype='str', encoding='utf-8', na_filter=False)
dfParams = dfParams.merge(dfPlans, how='inner', on='step_wid')
dfParams = dfParams[['name', 'value', 'type_cd', 'step_wid', 'datatype', 'context_type',
       'inactive_flag', 'comments', 'plan_name', 'step_name']]
dfParams = dfParams.drop_duplicates()
dfParams['param_filename'] = dfParams['plan_name'].apply(lambda x: re.sub(r'\W+', '_', x) + '.param')

# Lookup the Converted Mapping Task IDs

In [169]:
# Login to IDMC
logging.info('Logging into IDMC')
http = urllib3.PoolManager()

data = '{ "username": "' + config['idmc']['user'] + '", "password": "' + config['idmc']['password'] + '" }'

url = 'https://' + config['idmc']['host'] + '/saas/public/core/v3/login'
r = http.request(
    'POST', 
    url,
    timeout=3000,
    body=data,
    headers={
            'Accept': 'application/json',
            'Content-Type': 'application/json'
        }
    )
    
# Convert the response into a datframe
result = json.loads(r.data.decode('utf-8'))
sessionID = result['userInfo']['sessionId']


In [170]:
# Get the secure agent ID
logging.info('Getting the secure agent ID')
url = 'https://' + config['idmc']['pod'] + '.' + config['idmc']['host'] + '/saas/api/v2/runtimeEnvironment/name/' + quote(config['idmc']['agentGroupName'])
r = http.request(
    'GET', 
    url,
    timeout=3000,
    headers={
            'Accept': 'application/json',
            'icSessionId': sessionID
        }
    )
    
# Convert the response into a datframe
result = json.loads(r.data.decode('utf-8'))
agentGroupID = result['id']
agentGroupGUID = result['federatedId']
agentGroupName = config['idmc']['agentGroupName']

In [171]:
# Initialise the tasks data frame
logging.info('Getting a list of the mapping tasks')
limit = 200
i = 0
dfTasks = pd.DataFrame()

# Page through mapping task queries
while True:

    # Get a list of the mapping tasks
    url = 'https://' + config['idmc']['pod'] + '.' + config['idmc']['host'] + '/saas/public/core/v3/objects?q=type==%27MTT%27&limit=' + str(limit) + '&skip=' + str(i)
    r = http.request(
        'GET', 
        url,
        timeout=3000,
        headers={
                'Accept': 'application/json',
                'INFA-SESSION-ID': sessionID
            }
        )
        
    # Convert the response into a datframe
    result = json.loads(r.data.decode('utf-8'))
    dfTmp = pd.json_normalize(result)
    dfResp = dfTmp.copy()
    taskCount = dfResp.iloc[0]['count']
    dfResp = dfResp['objects'].explode()
    dfResp = pd.DataFrame(dfResp)
    dfResp = pd.json_normalize(dfResp['objects'])
    dfTasks = pd.concat([dfTasks, dfResp], ignore_index=True)

    # Break if all records have been returned
    i = i + limit
    if i > taskCount:
        print(f'Finished. dfCount = { len(dfTasks) }; i = { i }; taskCount = { taskCount }')
        break

    print(f'Looping. dfCount = { len(dfTasks) }; i = { i }; taskCount = { taskCount }')

# Clean up the list of returned tasks
dfTasks = dfTasks.drop(columns=['tags'])
dfTasks = dfTasks.drop_duplicates()



Looping. dfCount = 200; i = 200; taskCount = 299
Finished. dfCount = 299; i = 400; taskCount = 299


In [172]:
# Join the IDMC info onto the plans
dfTasks['step_name'] = dfTasks['path'].apply(lambda x: os.path.basename(x))
dfTasks = dfTasks[['step_name','id','path']]
dfTasks = dfTasks.rename(columns={'id': 'infa_id', 'path': 'infa_path'})
dfPlans = dfPlans.merge(dfTasks, how='left', on='step_name')
dfPlans['agent_id'] = agentGroupID
dfPlans['agent_guid'] = agentGroupGUID
dfPlans['agent_name'] = agentGroupName
dfPlans['script_dir'] = config['local']['scriptsDir']
dfPlans['script_args'] = '' # TODO placeholder for any args that need to be passed to the step script

In [173]:
# Log an error if any plans did not find an existing matching task
dfMissing = dfPlans[(dfPlans['infa_id'] == '') | (dfPlans['infa_id'].isna())].copy()
if len(dfMissing.index) > 0:
    logging.error('Some plans are missing a converted mapping task. Please see "out/missing_tasks.csv" for more details')
    dfMissing.to_csv('out/missing_tasks.csv', index=False, quoting=csv.QUOTE_ALL)

# Generate the Parameter Files

In [174]:
logging.info('Generating the parameter files...')
unique_files = dfParams['param_filename'].unique()

for filename in unique_files:
    
    logging.info(f'Generating the parameter file "out/{ filename }"...')
    unique_sections = dfParams[dfParams['param_filename'] == filename]['step_name'].unique()

    # Set the param file header
    lines = ['#USE_SECTIONS', '']

    for section in unique_sections:

        dfSection = dfParams[( dfParams['param_filename'] == filename ) & ( dfParams['step_name'] == section )].copy()

        # Append the section header
        step_name = dfSection.iloc[0]["step_name"]
        lines.append(f'[Default].[{ step_name }]')

        # Add the parameter values
        for idx, row  in dfSection.iterrows():
            param_name = row['name']
            param_value = row['value']
            lines.append(f"{ param_name }={ param_value }")
        lines.append('')

    # Add the global footer
    lines.append('[Global]')
    lines.append('')

    with open(f'out/{ filename }', 'w', encoding='utf-8') as file:
        file.write('\n'.join(lines))


# Generate the Taskflows

In [ ]:
logging.info('Generating the taskflows...')
planIds = dfPlans['plan_wid'].unique()

for planId in planIds:

    # Get the plan for the current id
    dfPlan = dfPlans[dfPlans['plan_wid'] == planId].copy()

    # Generate the unique identifiers
    dfPlan['dac2idmc_step_id'] = dfPlan.apply(lambda x: "task" + shortuuid.uuid()[:8], axis=1)
    map_order_uuid = { step_order: "group" + shortuuid.uuid()[:8] for step_order in dfPlan['plan_step_order'].unique() }
    dfPlan = dfPlan.sort_values(['plan_wid', 'plan_step_order', 'step_name'])
    dfPlan['sequence'] = range(len(dfPlan))
    dfPlan['dac2idmc_group_id'] = dfPlan['plan_step_order'].map(map_order_uuid)
    dfPlan['dac2idmc_next_id'] = dfPlan['dac2idmc_step_id'].shift(-1)

    # Get the next group IDs and sort the plans
    dfGroups = dfPlan[['plan_step_order','dac2idmc_group_id']].copy()
    dfGroups = dfGroups.drop_duplicates()
    dfGroups = dfGroups.sort_values(['plan_step_order'])
    dfGroups['dac2idmc_next_group'] = dfGroups['dac2idmc_group_id'].shift(-1)
    dfGroups = dfGroups[['dac2idmc_group_id','dac2idmc_next_group']]

    dfPlan = dfPlan.merge(dfGroups, how='inner', on='dac2idmc_group_id')
    dfPlan = dfPlan.sort_values(['sequence'])

    #TODO Uncomment the below lines for troubleshooting
    #print(planId)
    #dfPlan.to_excel(f'out/{ planId }.xlsx')

    # Generate the taskflow ID
    taskflowID = shortuuid.uuid()
    taskflowName = dfPlan.iloc[0]['plan_name']
    taskflowName = re.sub(r'\W+', '_', taskflowName)

    logging.info(f'Create workspace tree "tmp/{ taskflowName }/Explore/Default"')

    # Create the workspace directories
    treePath = Path(f'tmp/{ taskflowName }/Explore/Default')
    treePath.mkdir(parents=True, exist_ok=True)

    logging.info(f'Generating taskflow "{ taskflowName }"')
    generate_taskflow.generate_taskflow(taskflowID, taskflowName, dfPlan, config)

    logging.info(f'Packaging taskflow "{ taskflowName }"')
    package_import.package_import(taskflowID, taskflowName, dfPlan)

    logging.info(f'Done!')

logging.info('All taskflows have been generated!')

D3CA457A65BED1FA6E627DF045DFC5A1
74CB216C9DA89FAC19D2A05895E89F74
201C9CC7C59D167A79A2E247C6189A67
4C7EB9C8BB06489745D4C93191C1EF0
88898EF341B9D0DF4C39965380BD566
D1EE996AB7E780BDB2FDC9BCEDB3CB4C


# Testing below this line

In [ ]:
dfTasks[dfTasks['step_name'] == '']

,step_name,infa_id,infa_path
0,Load into Geography Dimension,4o6rpkCrceZiaOCY5g88eY,Default/Load into Geography Dimension
1,Load into Position Dimension,9ArGrsQE0F6jWzNaGutI0a,Default/Load into Position Dimension
2,Load into Product Dimension,33MaHQmgquFlHadf7pJM4W,Default/Load into Product Dimension
3,mp_ext_RAW_Key,an4lj4o3X5HdQ7dxzJ94eP,Customers/Bunnings/mp_ext_RAW_Key
4,mp_Simultaneous,7mSYo2bLKJwkBtbvHiV3ua,Default/mp_Simultaneous
...,...,...,...
294,TASK_GROUP_Load_PositionDimension,2V63mNQ1pKOhPGebjdsBw0,Default/TASK_GROUP_Load_PositionDimension
295,TASK_GROUP_Load_PositionHierarchy,3mW2lCshevukLlOW2OfTMQ,Default/TASK_GROUP_Load_PositionHierarchy
296,TASK_GROUP_Pre_Load_Positionhierarchy,1FGrllzaPSDdLjiBYziuAL,Default/TASK_GROUP_Pre_Load_Positionhierarchy
297,TC_ValueTest_NoSample__MCT__1692288033153,iBQsmsvfrcWd1gixaRh0Ur,Default/TC_ValueTest_NoSample__MCT__1692288033153
